In [3]:
import urlparse
import requests
import os

## Configuration

Get endpoint and authentication info from environment variables.

In [6]:
proto = os.environ.get('WES_API_PROTO')
host = os.environ.get('WES_API_HOST')
auth = os.environ.get('WES_API_AUTH')

In [9]:
headers = {'Authorization': auth}
endpoint = '{}://{}/ga4gh/wes/v1'.format(proto, host)

Create a simple object to store session/connection info.

In [12]:
class WorkflowServiceSession(object):
    """
    API session with workflow execution service endpoint.
    """
    def __init__(self, host, auth, proto='http'):
        self.endpoint = '{}://{}/ga4gh/wes/v1'.format(proto, host)
        self.headers = headers = {'Authorization': auth}


In [13]:
wes = WorkflowServiceSession(host=host, auth=auth, proto=proto)

## Getting info from WES

Create a generic `GET` request function (to minimize repeated code).

In [39]:
def wes_get_request(wes, target):
    """
    Generic workflow execution service API 'GET' request.
    """
    res = requests.get(
        '{}{}'.format(wes.endpoint, target),
        headers=wes.headers
    )
    return res.json()

Use `get_info` to retrieve summary information about the WES endpoint.

In [41]:
def get_info(wes):
    """
    Collect properties of a workflow execution service server endpoint.
    """
    target = '/service-info'
    return wes_get_request(wes, target)

In [42]:
get_info(wes)

{u'engine_versions': {u'cwltool': u'1.0.20171107133715',
  u'dockstore': u'1.3.0'},
 u'key_values': {u'flavour': u'e.g. (r2.medium), instance type descriptor for AWS'},
 u'supported_filesystem_protocols': [u'http',
  u'https',
  u'sftp',
  u's3',
  u'gs',
  u'synapse'],
 u'supported_wes_versions': [u'v1.0'],
 u'system_state_counts': {u'Complete': 21},
 u'workflow_type_versions': {u'cwl': {u'workflow_type_version': [u'1.0']},
  u'wdl': {u'workflow_type_version': [u'1.0']}}}

Use `list_workflow_runs` to list all workflow instances/runs/jobs (completed, initialized, or running) that have been recorded in the history of the WES endpoint.

In [43]:
def list_workflow_runs(wes, page_size=100, page_token=0):
    """
    List all workflow runs in workflow execution service server endpoint.
    """
    target = '/workflows'
    return wes_get_request(wes, target)
    

In [44]:
list_workflow_runs(wes)

{u'next_page_token': None,
 u'workflows': [{u'state': u'Complete', u'workflow_id': u'1'},
  {u'state': u'Complete', u'workflow_id': u'2'},
  {u'state': u'Complete', u'workflow_id': u'3'},
  {u'state': u'Complete', u'workflow_id': u'4'},
  {u'state': u'Complete', u'workflow_id': u'5'},
  {u'state': u'Complete', u'workflow_id': u'6'},
  {u'state': u'Complete', u'workflow_id': u'7'},
  {u'state': u'Complete', u'workflow_id': u'8'},
  {u'state': u'Complete', u'workflow_id': u'9'},
  {u'state': u'Complete', u'workflow_id': u'10'},
  {u'state': u'Complete', u'workflow_id': u'11'},
  {u'state': u'Complete', u'workflow_id': u'12'},
  {u'state': u'Complete', u'workflow_id': u'13'},
  {u'state': u'Complete', u'workflow_id': u'14'},
  {u'state': u'Complete', u'workflow_id': u'15'},
  {u'state': u'Complete', u'workflow_id': u'16'},
  {u'state': u'Complete', u'workflow_id': u'17'},
  {u'state': u'Complete', u'workflow_id': u'18'},
  {u'state': u'Complete', u'workflow_id': u'19'},
  {u'state': u'Com

Use `get_workflow_run` to retrieve details about a specific workflow run by its ID.

In [45]:
def get_workflow_run(wes, workflow_run_id):
    target = '/workflows/{}'.format(workflow_run_id)
    return wes_get_request(wes, target)

In [46]:
get_workflow_run(wes, '21')

{u'outputs': [{u'location': None,
   u'name': None,
   u'type': None,
   u'value': None}],
 u'request': {u'key_values': None,
  u'workflow_descriptor': u'#!/usr/bin/env cwl-runner\n\nclass: CommandLineTool\nid: "BAMStats"\nlabel: "BAMStats tool"\ncwlVersion: v1.0 \ndescription: |\n    ![build_status](https://quay.io/repository/collaboratory/dockstore-tool-bamstats/status)\n    A Docker container for the BAMStats command. See the [BAMStats](http://bamstats.sourceforge.net/) website for more information.\n\ndct:creator:\n  "@id": "http://orcid.org/0000-0002-7681-6415"\n  foaf:name: Brian O\'Connor\n  foaf:mbox: "mailto:briandoconnor@gmail.com"\n\nrequirements:\n  - class: DockerRequirement\n    dockerPull: "quay.io/collaboratory/dockstore-tool-bamstats:1.25-6_1.0"\n\nhints:\n  - class: ResourceRequirement\n    coresMin: 1\n    ramMin: 4092\n    outdirMin: 512000\n    doc: "the process requires at least 4G of RAM"\n\ninputs:\n  mem_gb:\n    type: int\n    default: 4\n    doc: "The memory,

Use `get_workflow_run_status` to retrieve the status of a workflow run (this is the same as the `state` property of a workflow run object). 

In [47]:
def get_workflow_run_status(wes, workflow_run_id):
    target = '/workflows/{}/status'.format(workflow_run_id)
    return wes_get_request(wes, target)

In [48]:
get_workflow_run_status(wes, '21')

{u'state': u'Complete', u'workflow_id': u'21'}

## Submitting workflows

Use `build_workflow_run_packet` to collect and format parameters/instructions for a new workflow run.

In [49]:
def build_workflow_run_packet(
    descriptor, params, workflow_type, workflow_type_version, key_values
):
    """
    Build JSON request packet for submitting a workflow run.
    """
    return {
        'workflow_descriptor': descriptor, 
        'workflow_params' : params, 
        'workflow_type': workflow_type,
        'workflow_type_version': workflow_type_version, 
        'key_values' : key_values
    }
    

`read_file` is a small helper function from Abraham (used here to read in JSON parameters file).

In [51]:
def read_file(file_name):
    file_r = open(file_name,'r')
    return file_r.read()

Example workflow from Abraham's gist/notebook (using the full descriptor URL instead of the Dockstore ID). Check out his notebook for the contents of `bams.json` (I just copied and pasted into a local file).

In [52]:
# Dockstore tool URL
descriptor = 'https://dockstore.org:8443/api/ga4gh/v2/tools/quay.io%2Fcollaboratory%2Fdockstore-tool-bamstats/versions/1.25-6_1.0/plain-CWL/descriptor/%2FDockstore.cwl"'

# File in local path
params = read_file('bams.json') 

workflow_type = 'cwl'
workflow_type_version = '1.0'
key_values = {'flavour' : 'c4.xlarge'} # Consonance specific key value

In [54]:
workflow_run_packet = build_workflow_run_packet(
    descriptor=descriptor,
    params=params,
    workflow_type=workflow_type,
    workflow_type_version=workflow_type_version,
    key_values=key_values
)

Create a generic `POST` request function (to minimize repeated code). Returns the response of the request.

In [40]:
def wes_post_request(wes, target, json):
    """
    Generic workflow execution service API 'POST' request.
    """
    res = requests.post(
        '{}{}'.format(wes.endpoint, target),
        headers=wes.headers,
        json = json
    )
    return res.json()

Use `submit_workflow_run` to initiate a new workflow run with the specified parameters. The response for this request will give the ID of the submitted workflow run.

In [55]:
def submit_workflow_run(wes, workflow_run_packet):
    target = '/workflows'
    return wes_post_request(wes, target, workflow_run_packet)

In [67]:
submit_workflow_run(wes, workflow_run_packet)

{u'workflow_id': u'21'}

**NOTE:** I think there's a bug with the Consonance workflow run counter (or the `POST` endpoint in WES) — we saw above that there were already 21 workflow runs, so the new run should have an ID corresponding to a larger number. Can double check:

In [69]:
list_workflow_runs(wes)

{u'next_page_token': None,
 u'workflows': [{u'state': u'Complete', u'workflow_id': u'1'},
  {u'state': u'Complete', u'workflow_id': u'2'},
  {u'state': u'Complete', u'workflow_id': u'3'},
  {u'state': u'Complete', u'workflow_id': u'4'},
  {u'state': u'Complete', u'workflow_id': u'5'},
  {u'state': u'Complete', u'workflow_id': u'6'},
  {u'state': u'Complete', u'workflow_id': u'7'},
  {u'state': u'Complete', u'workflow_id': u'8'},
  {u'state': u'Complete', u'workflow_id': u'9'},
  {u'state': u'Complete', u'workflow_id': u'10'},
  {u'state': u'Complete', u'workflow_id': u'11'},
  {u'state': u'Complete', u'workflow_id': u'12'},
  {u'state': u'Complete', u'workflow_id': u'13'},
  {u'state': u'Complete', u'workflow_id': u'14'},
  {u'state': u'Complete', u'workflow_id': u'15'},
  {u'state': u'Complete', u'workflow_id': u'16'},
  {u'state': u'Complete', u'workflow_id': u'17'},
  {u'state': u'Complete', u'workflow_id': u'18'},
  {u'state': u'Complete', u'workflow_id': u'19'},
  {u'state': u'Com

OK, so the new workflow run is actually `23`. Check status and details using the same functions as above.

In [72]:
get_workflow_run_status(wes, '23')

{u'state': u'Complete', u'workflow_id': u'23'}

In [73]:
get_workflow_run(wes, '23')

{u'outputs': [{u'location': None,
   u'name': None,
   u'type': None,
   u'value': None}],
 u'request': {u'key_values': None,
  u'workflow_descriptor': None,
  u'workflow_params': u'{\n  "bam_input": {\n        "class": "File",\n        "format": "http://edamontology.org/format_2572",\n        "path": "https://s3-us-west-2.amazonaws.com/achave11-redwood-storage/consonance-outputs/rna.SRR948778.bam"\n    },\n    "bamstats_report": {\n        "class": "File",\n        "path": "/tmp/bamstats_report.zip"\n    }\n}\n',
  u'workflow_type': u'cwl',
  u'workflow_type_version': u'1.0'},
 u'state': u'Complete',
 u'task_logs': [{u'cmd': None,
   u'endTime': u'2018-04-23 23:24:51.618324',
   u'exitCode': None,
   u'name': u'cwl',
   u'startTime': u'2018-04-24 22:12:24.934',
   u'stderr': u"/usr/local/bin/cwltool 1.0.20160712154127\n['docker', 'pull', 'quay.io/collaboratory/dockstore-tool-bamstats:1.25-6_1.0']\n1.25-6_1.0: Pulling from collaboratory/dockstore-tool-bamstats\n3f992ab3df53: Pulling fs